In [1]:
import os
import ssl
from pyesgf.logon import LogonManager
from pyesgf.search import SearchConnection

In [14]:
# Ensure that these environment variables are set, e.g. in your .bashrc
USERNAME = os.environ.get('ESGF_USERNAME')
PASSWORD = os.environ.get('ESGF_PASSWORD')

# login to ESGF and generate SSL context
myproxy_host = 'esgf-node.ipsl.upmc.fr'

lm = LogonManager()
lm.logon(username=USERNAME, password=PASSWORD, hostname=myproxy_host)

sslcontext = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
sslcontext.load_verify_locations(capath=lm.esgf_certs_dir)
sslcontext.load_cert_chain(lm.esgf_credentials)

# check if logged in
lm.is_logged_on()

True

In [ ]:
# define your query, eg:
query = {
    'project': 'CMIP6',
    'activity_drs': 'ScenarioMIP',
    'experiment_id': 'ssp534-over',
    'variable': 'tas',
    'frequency': 'mon',
}

# check ESGF for number of datasets that satisfy query
conn = SearchConnection('http://esgf-data.dkrz.de/esg-search', distrib=True)
context = conn.new_context(**query, facets=query.keys())
context.hit_count

61

In [17]:
# generate results and check datasets to verify all is working as expected
results = context.search()
for result in results:
    print(result.dataset_id)

CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp534-over.r1i1p1f1.Amon.tas.gn.v20210811|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp534-over.r1i1p1f1.Amon.tas.gn.v20190222|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp534-over.r1i1p1f1.Amon.tas.gn.v20191108|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.NCAR.CESM2-WACCM.ssp534-over.r1i1p1f1.Amon.tas.gn.v20200701|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.NCAR.CESM2-WACCM.ssp534-over.r2i1p1f1.Amon.tas.gn.v20200211|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp534-over.r8i1p1f2.Amon.tas.gn.v20191104|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp534-over.r101i1p1f1.Amon.tas.gr.v20200412|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp534-over.r2i1p1f2.Amon.tas.gr.v20190410|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp534-over.r3i1p1f2.Amon.tas.gr.v20190410|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp534-over.r4i1p1f2.Amon.tas.gr.v20190410|esgf.ceda.ac.uk
CMIP6.ScenarioMIP.CNRM-CERF

In [18]:
# get most recent dataset and print download urls
latest = list(results)[-1]
files = latest.file_context().search(ignore_facet_check=True)
for file in files:
    print(file.download_url)

https://esgf3.dkrz.de/thredds/fileServer/cmip6/ScenarioMIP/AWI/AWI-ESM-1-REcoM/ssp534-over/r1i1p1f1/Amon/tas/gn/v20230314/tas_Amon_AWI-ESM-1-REcoM_ssp534-over_r1i1p1f1_gn_204001-205412.nc
https://esgf3.dkrz.de/thredds/fileServer/cmip6/ScenarioMIP/AWI/AWI-ESM-1-REcoM/ssp534-over/r1i1p1f1/Amon/tas/gn/v20230314/tas_Amon_AWI-ESM-1-REcoM_ssp534-over_r1i1p1f1_gn_205501-207412.nc
https://esgf3.dkrz.de/thredds/fileServer/cmip6/ScenarioMIP/AWI/AWI-ESM-1-REcoM/ssp534-over/r1i1p1f1/Amon/tas/gn/v20230314/tas_Amon_AWI-ESM-1-REcoM_ssp534-over_r1i1p1f1_gn_207501-209412.nc
https://esgf3.dkrz.de/thredds/fileServer/cmip6/ScenarioMIP/AWI/AWI-ESM-1-REcoM/ssp534-over/r1i1p1f1/Amon/tas/gn/v20230314/tas_Amon_AWI-ESM-1-REcoM_ssp534-over_r1i1p1f1_gn_209501-210012.nc
